In [ ]:
# for other action
# construct input for tool 2 from tool 1's output
folder_name_1 = '/Users/kavyasrinet/Github/minecraft/python/craftassist/text_to_tree_tool/turk_data/' + \
                'combined_dicts/humanbot/other_actions/'
folder_name_2 = '/Users/kavyasrinet/Github/minecraft/python/craftassist/text_to_tree_tool/turk_data/' + \
                'combined_dicts/humanbot/other_actions/'

# tool_1_out_file = folder_name_1 + 'all_agreements.txt'
# tool2_input_file = folder_name_2 + 'input.txt'

tool_1_out_file = folder_name_1 + 'all_other_actions.txt'
tool2_input_file = folder_name_2 + 'all_other_tool_2_input.txt'
from pprint import pprint
'''
command: Input.command

'''
import copy
example_d = {}

def merge_indices(indices):
    a, b = indices[0]
    for i in range(1, len(indices)):
        x, y = indices[i]
        if x != b + 1:
            return indices
            # raise NotImplementedError("Spans not continuous during merging!!!")
        else:
            b = y
    if a==b:
        return a
    return [a, b]

import ast
import json


all_actions = {}
with open(tool_1_out_file, "r") as f1, open(tool2_input_file, 'w') as f2:
    composites= []
    
    for line in f1.readlines():
        line  = line.strip()
        cmd, out = line.split("\t")
        words = cmd.split()
        
        words_copy = copy.deepcopy(words)
        #pprint(out)
        action_dict = ast.literal_eval(out)
        action_type = action_dict['action_type'][1]
            
        # find children that need to be re-annotated
        for key, val in action_dict.items():
            child_name = None
            # child needs annotation
            if val[0]== 'no':
                # insert "span"
                words_copy = copy.deepcopy(words)
                child_name = key
                
                write_line = ""
                write_line += " ".join(words) + "\t"
                #print(words, child_name, action_type, val[1])
                
                indices = merge_indices(val[1])
                span_text = None
                if type(indices) == list:
                    if type(indices[0]) == list:
                        # this means that indices were scattered and disjoint
                        for idx in indices:
                            words_copy[idx[0]] = "<span style='background-color: #FFFF00'>" + words_copy[idx[0]]
                            words_copy[idx[1]] = words_copy[idx[1]] + "</span>"
                    else:
                        words_copy[indices[0]] = "<span style='background-color: #FFFF00'>" + words_copy[indices[0]]
                        words_copy[indices[1]] = words_copy[indices[1]] + "</span>"
                else:
                    words_copy[indices] = "<span style='background-color: #FFFF00'>" + words_copy[indices] + "</span>"
                write_line += " ".join(words_copy) + "\t" + action_type + "\t" + child_name
                f2.write(write_line+ "\n")
                #print(write_line)
        
        #print("----"*5)



In [ ]:
# combining for otheraction
# output of tool 1
folder = '/Users/kavyasrinet/Github/minecraft/python/craftassist/text_to_tree_tool/turk_data/combined_dicts/humanbot/other_actions/'
tool_1_out_file = folder + 'tool1_out.txt'
# output of tool 2
tool_2_out_file = folder + 'all_agreements.txt'

# combine outputs
# check if all keys of t1 annotated yes -> put directly
# if no , check child in t2 and combine
# construct mape of tool 1
tool1_map = {}
with open(tool_1_out_file) as f:
    for line in f.readlines():
        line = line.strip()
        cmd, a_d = line.split("\t")
        tool1_map[cmd]= a_d

# construct map of tool 2
tool2_map = {}
with open(tool_2_out_file) as f:
    for line in f.readlines():
        line = line.strip()
        cmd, child, child_dict = line.split("\t")
        if cmd in tool2_map and child in tool2_map[cmd]:
            print("BUGGG")
        if cmd not in tool2_map:
            tool2_map[cmd] = {}
        tool2_map[cmd][child] = child_dict        

import ast
def all_yes(a_dict):
    if type(a_dict) == str:
        a_dict = ast.literal_eval(a_dict)
    for k, val in a_dict.items():
        if type(val) == list and val[0] == "no":
            return False
    return True

def clean_dict_1(a_dict):
    if type(a_dict) == str:
        a_dict = ast.literal_eval(a_dict)
    new_d = {}
    for k, val in a_dict.items():
        if type(val) == list:
            if val[0] in ["yes", "no"]:
                new_d[k] = val[1]                
        elif type(val) == dict:
            new_d[k] = a_dict(val[1])
        else:
            new_d[k] = val
    return new_d

i  =0
from pprint import pprint
# update dict of tool1 with tool 2
with open(folder + 'final/other_actions_resolved.txt', 'w') as f:
    for cmd, a_dict in tool1_map.items():
        clean_dict = clean_dict_1(a_dict)
        if all_yes(a_dict):
            f.write(cmd + "\t" + str(clean_dict) + "\n")
            #print(clean_dict)
            continue
        child_dict = tool2_map[cmd]
        for k, v in child_dict.items():
            if k not in clean_dict:
                print(cmd, k)
                print(a_dict)
                print(clean_dict)
                print(child_dict)
                print("BUGGGG")
            if type(v) == str:
                v = ast.literal_eval(v)
            #print(k, v)
            if not v:
                continue
            #print(cmd, a_dict, child_dict)
            if k == "tag_val":
                clean_dict.update(v)
            else:
                clean_dict[k] = v[k]
        valid_dict = {}
        valid_dict['dialogue_type'] = clean_dict['dialogue_type']
        del clean_dict['dialogue_type']
        #print(actual_dict['action_type'])
        clean_dict['action_type'] = clean_dict['action_type'].upper()
        valid_dict['action_sequence'] = [clean_dict]
        f.write(cmd + "\t" + str(valid_dict) + "\n")
#     pprint(clean_dict)
#     print("*"*20)
#     if i==20:
#         break


In [1]:
# post_process spans and "contains_coreference" : "no"
def merge_indices(indices):
    a, b = indices[0]
    for i in range(1, len(indices)):
        a = min(a, indices[i][0])
        b = max(b, indices[i][1])
    return [a, b]

def fix_put_mem(d):
    
    if type(d) == str:
        d = ast.literal_eval(d)
    new_d = copy.deepcopy(d)
    del new_d['action_type']
    if 'has_tag' in new_d and 'upsert' in new_d:
        new_d['upsert']['memory_data']['has_tag'] = new_d['has_tag']
        del new_d['has_tag']

    return new_d
def fix_spans(d):
    new_d = {}
    if type(d) == str:
        d = ast.literal_eval(d)
    for k, v in d.items():
        if k == "contains_coreference" and v == "no":
            continue
        if type(v) == list:
            if k == "tag_val":
                new_d["has_tag"] = [0, merge_indices(v)] 
            else:
                new_d[k] = [0, merge_indices(v)] 
            continue
        elif type(v) == dict:
            new_d[k] = fix_spans(v)
            continue
        else:
            new_d[k] = v
    return new_d
        
i = 0
input_file = '../../minecraft/python/craftassist/text_to_tree_tool/turk_data/combined_dicts/humanbot/other_actions/final/other_actions_resolved.txt'
output_file = '../../minecraft/python/craftassist/text_to_tree_tool/turk_data/combined_dicts/humanbot/other_actions/final/other_actions_postprocessed.txt'
with open(input_file) as f, open(output_file, 'w') as f_w:
    for line in f.readlines():
        i+= 1
        line = line.strip()
        text, d = line.split("\t")
        d = ast.literal_eval(d)
        action_dict = fix_spans(d['action_sequence'][0])
        if action_dict['action_type'] == 'TAG':
            updated_dict = fix_put_mem(action_dict)
            new_d = {}
            new_d['dialogue_type'] = d['dialogue_type']
            new_d.update(updated_dict)
        elif action_dict['action_type'] == 'ANSWER':
            #print(d)
            new_d = {}
            new_d['dialogue_type'] = 'GET_MEMORY'
        else:
            if action_dict['action_type'] == 'COPY':
                action_dict['action_type'] = 'BUILD'
            d['action_sequence'] = [action_dict]
            new_d = d
        f_w.write(text+  "\t" + json.dumps(new_d) + "\n")
        

NameError: name 'ast' is not defined

In [2]:
# combine all prompts data
in1 = '../../minecraft/python/craftassist/text_to_tree_tool/turk_data/combined_dicts/humanbot/all_final_postprocessed.txt'
in2 = '../../minecraft/python/craftassist/text_to_tree_tool/turk_data/combined_dicts/humanbot/other_actions/final/other_actions_postprocessed.txt'
in3 = '../../minecraft/python/craftassist/text_to_tree_tool/turk_data/composites/humanbot_combined_postprocessed.txt'
out_file = '../../minecraft/python/craftassist/text_to_tree_tool/turk_data/final_prompts_and_humanbot/all_humanbot_data.txt'

all_text = []


1
2
3


In [26]:
import json
with open('../../minecraft/python/craftassist/text_to_tree_tool/turk_data/final_prompts_and_humanbot/all_humanbot_data.txt') as f, \
    open('../../minecraft/python/craftassist/text_to_tree_tool/turk_data/final_prompts_and_humanbot/all_humanbot_data_cp.txt', 'w') as fw:
    for line in f.readlines():
        line = line.strip().split("\t")
        #print(line)
        text, ad = line
        print(type(ad))
        x = json.dumps(ad)
        print(type(x))
        #fw.write(text + "\t" + x+ "\n")
        break

<class 'str'>
<class 'str'>


In [ ]:
with open('../../minecraft/python/craftassist/text_to_tree_tool/turk_data/combined_dicts/humanbot/all_final_postprocessed.txt') as f:
    

In [17]:
import ast
import json
data = []
with open('../../minecraft/python/craftassist/text_to_tree_tool/turk_data/final_prompts_and_humanbot/all_humanbot_data.txt') as f, \
 open('../../minecraft/python/craftassist/text_to_tree_tool/turk_data/final_prompts_and_humanbot/all_humanbot_final.json', 'w') as fw:
    for line in f.readlines():
        line  =  line.strip().split("\t")
        text, ad = line
        meh = json.loads(ad)
        dd = ast.literal_eval(meh)
        if dd['dialogue_type'] == 'HUMAN_GIVE_COMMAND' and len(dd['action_sequence']) == 1 and \
        dd['action_sequence'][0]['action_type'] == 'NOOP':
            cnt += 1
            dd_new = {'dialogue_type': 'NOOP'}
            data.append([[text], dd_new])
            continue
        else:
            data.append([[text], dd])
    json.dump(data, fw)

In [18]:
print(len(data))

2161


In [19]:
with open('../../minecraft/python/craftassist/text_to_tree_tool/turk_data/final_prompts_and_humanbot/all_humanbot_final.json') as fw:
    data_n = json.load(fw)

In [20]:
print(type(data_n))

<class 'list'>


In [21]:
type(data_n[0][1])

dict